# Tmap 검색건수 : 정답 레이블 전처리 

관광수요를 예측하는 LSTM 모델에 지도 학습에 쓰일 정답 레이블 Tmap 검색 건수 전처리를 진행함.

[데이터셋 변수 설명] 
- 순위 = 관광지 인기 순위
- 관광지명 = 관광객의 목적지
- 도로명주소 = 관광지명 주소
- 중분류 카테고리 = 중분류로 관광지명 유형분류
- 소분류 카테고리 = 소분류로 관광지명 유형분류
- 검색건수 = 관광지명 Tmap에서 검색된 건수

[전처리 과정 설명] 
- 일간으로 다운받은 365개의 Tmap건수 데이터 불러옴. 
- 필요한 열 '관광지명', '검색건수' 열만 남기고 불필요한 열 drop
- 한국관광데이터랩에서는 검색 건수 데이터를 기간별로 상위 500개만을 제공하는데, 365일 모든 일자에 대해 순위 안에 드는 데이터를 혼잡한 관광지로 판단하여 하나의 리스트로 저장함
- 혼잡한 관광지에 대해 2021년의 데이터를 일자별로 추출하여 타겟 데이터를 제외하여 가공한 데이터셋에 조인함


[기타 데이터 관련 설명] 
- 자동차 이용 방문자의 이동행태이며 대중교통 및 도보 이용 방문자의 특성과 다를 수 있음.

source : 한국관광 데이터랩 - 빅데이터 - 내비게이션 - 지역별 관광지 검색순위
https://datalab.visitkorea.or.kr/datalab/portal/bda/getTourVisitCnt.do#

In [1]:
import pandas as pd
from glob import glob

In [2]:
#glob()로 서귀포시 티맵검색 데이터 불러오기
file_names = glob("../Dataset/raw_data/target/target_s/*.xlsx")
tmap_seogwipo = pd.DataFrame() 


for file_name in file_names:
    tmap = pd.read_excel(file_name)
    #temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'minimum temperature'}, inplace=True)

    tmap_seogwipo = pd.concat([tmap_seogwipo, tmap])


tmap_seogwipo

,순위,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,1,서귀포매일올레시장,제주 서귀포시 중앙로62번길 18-0,쇼핑,시장,543
1,2,성산일출봉,제주 서귀포시 일출로 284-12,자연관광,자연경관(산),301
2,3,섭지코지,제주 서귀포시 섭지코지로 107-0,기타관광,기타관광,231
3,4,오설록티뮤지엄,제주 서귀포시 신화역사로 15-0,문화관광,전시시설,177
4,5,동백포레스트,제주 서귀포시 생기악로 53-38,체험관광,농/산/어촌체험,168
...,...,...,...,...,...,...
530,493,미향해장국중문점,제주 서귀포시 일주서로 962-0,음식,한식,10
531,493,돈사돈서귀포점,제주 서귀포시 이어도로 778-1,음식,한식,10
532,493,초콜릿랜드,제주 서귀포시 중문관광로110번길 15-0,문화관광,전시시설,10
533,493,봉주르마담,제주 서귀포시 대청로 33-0,음식,간이음식,10


In [3]:
#필요한 열만 추출
tmap_seogwipo = tmap_seogwipo[['관광지명', '검색건수']]
tmap_seogwipo

,관광지명,검색건수
0,서귀포매일올레시장,543
1,성산일출봉,301
2,섭지코지,231
3,오설록티뮤지엄,177
4,동백포레스트,168
...,...,...
530,미향해장국중문점,10
531,돈사돈서귀포점,10
532,초콜릿랜드,10
533,봉주르마담,10


In [4]:
#제주도 poi 데이터 불러오기
poi_list = pd.read_excel("../Dataset/final_data/Jeju_POI_withexplanation_final.xlsx", header=0)
poi_list = poi_list[['Unnamed: 1','Unnamed: 2']]
poi_list.columns = ['poi','location']
# poi_list=poi_list[['poi','location']]
poi_list

,poi,location
0,헬로키티아일랜드,서귀포시
1,엉알해안산책로,제주시
2,1100고지습지,서귀포시
3,우도도항선선착장,제주시
4,혼인지,서귀포시
...,...,...
116,제주불빛정원,제주시
117,더마파크,제주시
118,거문오름,제주시
119,다랑쉬오름,제주시


In [5]:
#서귀포시 poi만 추출 
poi_seogwipo= poi_list[poi_list['location'] == '서귀포시']
poi_seogwipo=poi_seogwipo['poi']
poi_seogwipo

0          헬로키티아일랜드
2          1100고지습지
4               혼인지
5              안덕계곡
6          제주 초콜릿랜드
           ...     
105       휴애리자연생활공원
106       제주항공우주박물관
110           카멜리아힐
113    세계자동차&피아노박물관
115         서귀포치유의숲
Name: poi, Length: 62, dtype: object

In [6]:
#서귀포시의 poi list 형태로 반환 
poi_seogwipo_val = poi_seogwipo.values
poi_seogwipo_list = poi_seogwipo_val.tolist()
poi_seogwipo_list

['헬로키티아일랜드',
 '1100고지습지',
 '혼인지',
 '안덕계곡',
 '제주 초콜릿랜드',
 '여미지식물원',
 '본태박물관',
 '쇠소깍',
 '원앙폭포',
 '천지연폭포 ',
 '박물관은 살아있다 ',
 '남원큰엉',
 '군산오름',
 '붉은오름자연휴양림',
 '초콜릿박물관',
 '중문관광단지',
 '용머리해안',
 '제주민속촌',
 '천제연폭포',
 '정방폭포',
 '성산일출봉',
 '알뜨르비행장',
 '서귀포잠수함',
 '조랑말체험공원',
 '섭지코지',
 '우도잠수함',
 '엉또폭포',
 '화순금모래 해변',
 '아쿠아플라넷제주',
 '제주허브동산',
 '외돌개',
 '신양섭지해수욕장',
 '황우지해안',
 '테디베어뮤지엄제주',
 '약천사',
 '주상절리대',
 '성읍민속마을',
 '건강과성박물관',
 '산방굴사',
 '서귀포매일올레시장',
 '가파도',
 '걸매생태공원',
 '일출랜드',
 '김영갑갤러리두모악',
 '소정방폭포',
 '제주조각공원',
 '산방산탄산온천',
 '퍼시픽리솜',
 '점보빌리지',
 '선녀와나무꾼테마공원',
 '물영아리오름',
 '산방산',
 '제주다원녹차미로공원',
 '제이엠그랑블루요트',
 '따라비오름',
 '피규어뮤지엄제주',
 '광치기해변',
 '휴애리자연생활공원',
 '제주항공우주박물관',
 '카멜리아힐',
 '세계자동차&피아노박물관',
 '서귀포치유의숲']

In [7]:
#서귀포시 poi리스트만을 대상으로 하여 티맵 검색 건수 데이터 색인
tmap_seogwipo=tmap_seogwipo.loc[tmap_seogwipo['관광지명'].isin(poi_seogwipo_list)]
tmap_seogwipo

,관광지명,검색건수
0,서귀포매일올레시장,543
1,성산일출봉,301
2,섭지코지,231
6,광치기해변,160
7,카멜리아힐,158
...,...,...
338,물영아리오름,16
340,제주조각공원,16
388,산방굴사,14
440,안덕계곡,12


In [8]:
#혼잡한 서귀포시 poi확인
tmap_seogwipo['관광지명'].value_counts() == 365

서귀포매일올레시장        True
남원큰엉             True
산방산탄산온천          True
성산일출봉            True
붉은오름자연휴양림        True
제주항공우주박물관        True
세계자동차&피아노박물관     True
제주민속촌            True
건강과성박물관          True
본태박물관            True
천제연폭포            True
헬로키티아일랜드         True
외돌개              True
약천사              True
성읍민속마을           True
쇠소깍              True
섭지코지             True
정방폭포             True
용머리해안            True
중문관광단지           True
아쿠아플라넷제주         True
주상절리대            True
광치기해변            True
산방산              True
카멜리아힐            True
피규어뮤지엄제주        False
군산오름            False
황우지해안           False
휴애리자연생활공원       False
물영아리오름          False
서귀포치유의숲         False
테디베어뮤지엄제주       False
신양섭지해수욕장        False
안덕계곡            False
서귀포잠수함          False
제주허브동산          False
김영갑갤러리두모악       False
초콜릿박물관          False
따라비오름           False
엉또폭포            False
원앙폭포            False
점보빌리지           False
퍼시픽리솜           False
일출랜드            False
제주조각공원          False
알뜨르비행장    

In [9]:
#혼잡한 서귀포 poi만 출력
confusion_seogwipo = ['아쿠아플라넷제주','성산일출봉','쇠소깍','헬로키티아일랜드','건강과성박물관','외돌개','산방산','섭지코지',
'서귀포매일올레시장','산방산탄산온천','붉은오름자연휴양림','광치기해변''제주민속촌''남원큰엉''정방폭포','중문관광단지','본태박물관',
'약천사','성읍민속마을','세계자동차&피아노박물관','제주항공우주박물관','용머리해안','천제연폭포','카멜리아힐','주상절리대']

#for confusion_poi in confusion_jeju:
 #    df = tmap_jeju[tmap_jeju['관광지명'] == '한림공원']
  #   df = df['검색건수']
   #  df = df.reset_index()
    #temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'minimum temperature'}, inplace=True)

    #tmap_jeju = pd.concat([tmap_jeju, tmap])

In [10]:
#poi별로 index 재설정 : poi별로 조건을 변경하여 실행해야 함.

df = tmap_seogwipo[tmap_seogwipo['관광지명'] == '주상절리대']
df = df['검색건수']
df = df.reset_index()
df
#df = df.drop(['index'], axis=1, inplace=True)
#df
#df.to_csv('./Hallim_Park.csv',index=False)

,index,검색건수
0,17,91
1,11,114
2,8,75
3,6,83
4,6,83
...,...,...
360,11,171
361,10,177
362,10,190
363,17,168


In [143]:
#기존 데이터셋 불러오기
#파일은 poi 별로 불러와야 함.
df2 = pd.read_csv('./weather_final_with_buzz/서귀포시-주상절리대.csv')
df2

,visit,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type
0,주상절리대,2021-01-01,5,1,1,3,11,60,30,0
1,주상절리대,2021-01-02,8,2,0,5,10,70,20,0
2,주상절리대,2021-01-03,8,2,0,7,12,70,20,0
3,주상절리대,2021-01-04,8,0,0,3,12,60,0,0
4,주상절리대,2021-01-05,8,0,0,1,8,65,20,0
...,...,...,...,...,...,...,...,...,...,...
360,주상절리대,2021-12-27,7,0,0,9,11,65,20,0
361,주상절리대,2021-12-28,10,0,0,5,13,70,30,0
362,주상절리대,2021-12-29,7,0,0,3,9,60,30,0
363,주상절리대,2021-12-30,12,0,0,3,8,50,20,0


In [144]:
#기존 데이터셋과 정답레이블 병합
df = pd.concat([df2, df],axis=1)
df

,visit,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type,index,검색건수
0,주상절리대,2021-01-01,5,1,1,3,11,60,30,0,6,278
1,주상절리대,2021-01-02,8,2,0,5,10,70,20,0,10,320
2,주상절리대,2021-01-03,8,2,0,7,12,70,20,0,13,201
3,주상절리대,2021-01-04,8,0,0,3,12,60,0,0,5,182
4,주상절리대,2021-01-05,8,0,0,1,8,65,20,0,4,367
...,...,...,...,...,...,...,...,...,...,...,...,...
360,주상절리대,2021-12-27,7,0,0,9,11,65,20,0,9,293
361,주상절리대,2021-12-28,10,0,0,5,13,70,30,0,9,211
362,주상절리대,2021-12-29,7,0,0,3,9,60,30,0,12,261
363,주상절리대,2021-12-30,12,0,0,3,8,50,20,0,6,83


In [145]:
#칼러명 재정의
df.rename(columns={'visit' : 'poi',
                   '검색건수' : 'num_of_search'}, inplace=True)
df.drop(['index'], axis=1, inplace=True)
df

,poi,date,buzz,day,holiday,minimum temperature(°C),maximum temperature(°C),humidity(%),precip_prob(%),precip_type,num_of_search
0,주상절리대,2021-01-01,5,1,1,3,11,60,30,0,278
1,주상절리대,2021-01-02,8,2,0,5,10,70,20,0,320
2,주상절리대,2021-01-03,8,2,0,7,12,70,20,0,201
3,주상절리대,2021-01-04,8,0,0,3,12,60,0,0,182
4,주상절리대,2021-01-05,8,0,0,1,8,65,20,0,367
...,...,...,...,...,...,...,...,...,...,...,...
360,주상절리대,2021-12-27,7,0,0,9,11,65,20,0,293
361,주상절리대,2021-12-28,10,0,0,5,13,70,30,0,211
362,주상절리대,2021-12-29,7,0,0,3,9,60,30,0,261
363,주상절리대,2021-12-30,12,0,0,3,8,50,20,0,83


In [146]:
#csv로 저장
df.to_csv('./seogwipo_search/Jusangjeolli_Cliff.csv',index=False)